Step 1a: load the documents. In this case, I'm gonna start with the CSV file. 

In [1]:
#the function read_csv allows me to import a CSV file as a Pandas DataFrame

from pandas import read_csv

graph_publications = read_csv ("graph_publications.csv", keep_default_na=False,
                               dtype={
                                   "id": "string",
                                   "title": "string",
                                   "type":"string",
                                   "publication_year":"int",
                                   "issue": "string",
                                   "volume": "string",
                                   "chapter":"string",
                                   "publication_venue": "string",
                                   "venue_type": "string",
                                   "publisher":"string",
                                   "event":"string"
                               })
graph_publications

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1016/j.websem.2021.100655,Crossing The Chasm Between Ontology Engineerin...,journal-article,2021,,70,,Journal Of Web Semantics,journal,crossref:78,
1,doi:10.1007/s10115-017-1100-y,Core Techniques Of Question Answering Systems ...,journal-article,2017,3,55,,Knowledge And Information Systems,journal,crossref:297,
2,doi:10.1016/j.websem.2014.03.003,Api-Centric Linked Data Integration: The Open ...,journal-article,2014,,29,,Journal Of Web Semantics,journal,crossref:78,
3,doi:10.1093/nar/gkz997,The Monarch Initiative In 2019: An Integrative...,journal-article,2019,D1,48,,Nucleic Acids Research,journal,crossref:286,
4,doi:10.3390/publications7030050,Dras-Tic Linked Data: Evenly Distributing The ...,journal-article,2019,3,7,,Publications,journal,crossref:1968,
...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.1145/3407194,Early Detection Of Social Media Hoaxes At Scale,journal-article,2020,4,14,,Acm Transactions On The Web,journal,crossref:320,
496,doi:10.3390/app10144893,Cognitive Aspects-Based Short Text Representat...,journal-article,2020,14,10,,Applied Sciences,journal,crossref:1968,
497,doi:10.1145/3309547,Temporal Relational Ranking For Stock Prediction,journal-article,2019,2,37,,Acm Transactions On Information Systems,journal,crossref:320,
498,doi:10.1007/978-3-030-58285-2_27,Fast Pathfinding In Knowledge Graphs Using Wor...,book-chapter,2020,,,1,Lecture Notes In Computer Science - Ki 2020: A...,book,crossref:297,


Step 1b: load the json file

In [4]:
from json import load
from pandas import Series
from pandas import DataFrame

with open("graph_other_data.json", "r", encoding="utf-8") as f:
    graph_other_data = load(f)

graph_other_data_authors = graph_other_data.get("authors")
graph_other_data_venues = graph_other_data.get("venues_id")
graph_other_data_ref = graph_other_data.get("references")
graph_other_data_pub = graph_other_data.get("publishers")

#authors
auth_doi=[]
auth_fam=[]
auth_giv=[]
auth_id=[]

for key in graph_other_data_authors:
    a=[]
    b=[]
    c=[]
    auth_val = graph_other_data_authors[key]
    auth_doi.append(key)
    for dict in auth_val:
        a.append(dict["family"])
        b.append(dict["given"])
        c.append(dict["orcid"])
    auth_fam.append(a)
    auth_giv.append(b)
    auth_id.append(c)

    auth_doi_s=Series(auth_doi)
auth_fam_s=Series(auth_fam)
auth_giv_s=Series(auth_giv)
auth_id_s=Series(auth_id)

authors_df=DataFrame({
    "auth doi" : auth_doi_s,
    "family name" : auth_fam_s, 
    "given name" : auth_giv_s, 
    "orcid" : auth_id_s
})


#venues
venues_doi=[]
venues_issn=[]

for key in graph_other_data_venues:
    venues_val = graph_other_data_venues[key]
    venues_doi.append(key)
    venues_issn.append(venues_val)

unique_venues=[]
list_dois=[]

for i in range(len(venues_issn)):
    if venues_issn[i] not in unique_venues:
        unique_venues.append(venues_issn[i])
        dois = []
        dois.append(venues_doi[i])
        list_dois.append(dois)
    else:
        j = unique_venues.index(venues_issn[i])
        list_dois[j].append(venues_doi[i])

un_venues_doi_s=Series(list_dois)
un_venues_issn_s=Series(unique_venues)        
unique_venues_df=DataFrame({
    "venues doi" : un_venues_doi_s,
    "issn" : un_venues_issn_s, 
})

venues_ids = unique_venues_df[["venues doi", "issn"]]
venues_internal_id = []
for idx, row in venues_ids.iterrows():
    venues_internal_id.append("venues-" + str(idx))
venues_ids.insert(0, "venues id", Series(venues_internal_id, dtype="string"))
venues_id_intid=venues_ids.filter(["venues id"])
venues_id_intid=venues_id_intid.values.tolist()
venues_id_doi=venues_ids.filter(["venues doi"])
venues_id_doi=venues_id_doi.values.tolist()
venues_id_issn=venues_ids.filter(["issn"])
venues_id_issn=venues_id_issn.values.tolist()

l_intid=[]
l_doi=[]
l_issn=[]
for outlist in venues_id_doi:
    i=venues_id_doi.index(outlist)
    for list in outlist:
        for doi in list:
            l_doi.append(doi)
            l_intid.append(venues_id_intid[i][0])
            l_issn.append(venues_id_issn[i][0])
s_intid=Series(l_intid)
s_doi=Series(l_doi)
s_issn=Series(l_issn)
unique_venues_ids_df=DataFrame({"venues_intid":s_intid, "venues doi":s_doi, "issn":s_issn})

#references
ref_doi=[]
ref_cit=[]

for key in graph_other_data_ref:
    ref_val = graph_other_data_ref[key]
    ref_doi.append(key)
    ref_cit.append(ref_val)

ref_doi_s=Series(ref_doi)
ref_cit_s=Series(ref_cit)

ref_df=DataFrame({
    "ref doi" : ref_doi_s,
    "citation" : ref_cit_s, 
})

#publishers
pub_cr=[]
pub_id=[]
pub_name=[]

for key in graph_other_data_pub:
    pub_val = graph_other_data_pub[key]
    pub_cr.append(key)
    pub_id.append(pub_val["id"])
    pub_name.append(pub_val["name"])

pub_cr_s=Series(pub_cr)
pub_id_s=Series(pub_id)
pub_name_s=Series(pub_name)

publishers_df=DataFrame({
    "crossref" : pub_cr_s,
    "publisher id" : pub_id_s, 
    "name" : pub_name_s, 
})

unique_venues_ids_df

,venues_intid,venues doi,issn
0,venues-0,doi:10.1016/j.websem.2021.100655,[issn:1570-8268]
1,venues-0,doi:10.1016/j.websem.2014.03.003,[issn:1570-8268]
2,venues-0,doi:10.1016/j.websem.2014.06.002,[issn:1570-8268]
3,venues-0,doi:10.1016/j.websem.2021.100681,[issn:1570-8268]
4,venues-0,doi:10.1016/j.websem.2021.100658,[issn:1570-8268]
...,...,...,...
481,venues-304,doi:10.1007/978-3-030-59833-4_1,"[isbn:9783030598327, isbn:9783030598334]"
482,venues-305,doi:10.1515/jib-2018-0023,[issn:1613-4516]
483,venues-306,doi:10.1145/3407194,"[issn:1559-1131, issn:1559-114X]"
484,venues-307,doi:10.1145/3309547,"[issn:1046-8188, issn:1558-2868]"


Step 1c: merge the dataframes
<b style="font-size:30px">OUR FINAL DATAFRAME!!!</b>

In [6]:
from pandas import merge
#import pandas as pd

df_publications_auth_venues = merge(graph_publications, unique_venues_ids_df, left_on="id", right_on="venues doi", how="outer")
#df_publications_auth_venues = pd.concat([graph_publications, unique_venues_ids_df], axis=1)
df_publications_auth_venues

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event,venues_intid,venues doi,issn
0,doi:10.1016/j.websem.2021.100655,Crossing The Chasm Between Ontology Engineerin...,journal-article,2021,,70,,Journal Of Web Semantics,journal,crossref:78,,venues-0,doi:10.1016/j.websem.2021.100655,[issn:1570-8268]
1,doi:10.1007/s10115-017-1100-y,Core Techniques Of Question Answering Systems ...,journal-article,2017,3,55,,Knowledge And Information Systems,journal,crossref:297,,venues-1,doi:10.1007/s10115-017-1100-y,"[issn:0219-1377, issn:0219-3116]"
2,doi:10.1016/j.websem.2014.03.003,Api-Centric Linked Data Integration: The Open ...,journal-article,2014,,29,,Journal Of Web Semantics,journal,crossref:78,,venues-0,doi:10.1016/j.websem.2014.03.003,[issn:1570-8268]
3,doi:10.1093/nar/gkz997,The Monarch Initiative In 2019: An Integrative...,journal-article,2019,D1,48,,Nucleic Acids Research,journal,crossref:286,,venues-2,doi:10.1093/nar/gkz997,"[issn:0305-1048, issn:1362-4962]"
4,doi:10.3390/publications7030050,Dras-Tic Linked Data: Evenly Distributing The ...,journal-article,2019,3,7,,Publications,journal,crossref:1968,,venues-3,doi:10.3390/publications7030050,[issn:2304-6775]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.1145/3407194,Early Detection Of Social Media Hoaxes At Scale,journal-article,2020,4,14,,Acm Transactions On The Web,journal,crossref:320,,venues-306,doi:10.1145/3407194,"[issn:1559-1131, issn:1559-114X]"
496,doi:10.3390/app10144893,Cognitive Aspects-Based Short Text Representat...,journal-article,2020,14,10,,Applied Sciences,journal,crossref:1968,,venues-48,doi:10.3390/app10144893,[issn:2076-3417]
497,doi:10.1145/3309547,Temporal Relational Ranking For Stock Prediction,journal-article,2019,2,37,,Acm Transactions On Information Systems,journal,crossref:320,,venues-307,doi:10.1145/3309547,"[issn:1046-8188, issn:1558-2868]"
498,doi:10.1007/978-3-030-58285-2_27,Fast Pathfinding In Knowledge Graphs Using Wor...,book-chapter,2020,,,1,Lecture Notes In Computer Science - Ki 2020: A...,book,crossref:297,,venues-308,doi:10.1007/978-3-030-58285-2_27,"[isbn:9783030582845, isbn:9783030582852]"


Step 2: Create an empty Graph using the rdflib library

In [3]:
from rdflib import Graph
from rdflib import URIRef
from rdflib import Literal
from rdflib import RDF

my_graph = Graph()

Step 3: Define URIs. Each statement of the Graph will have the form Subject, Predicate, Object. Some of these elements are already defined in the rdflib library (e.g.: RDF.type is an already defined property, corresponding to the URIRef http://www.w3.org/1999/02/22-rdf-syntax-ns#type), the missing ones must be defined according to the UML model.

In [4]:
# some classes of resources (not finished yet)
JournalArticle = URIRef("https://schema.org/ScholarlyArticle")
BookChapter = URIRef("https://schema.org/Chapter")
Journal = URIRef("https://schema.org/Periodical")
Book = URIRef("https://schema.org/Book")

# some attributes related to classes (not finished yet)
doi = URIRef("https://schema.org/identifier")
publicationYear = URIRef("https://schema.org/datePublished")
title = URIRef("https://schema.org/name")
issue = URIRef("https://schema.org/issueNumber")
volume = URIRef("https://schema.org/volumeNumber")
identifier = URIRef("https://schema.org/identifier")
name = URIRef("https://schema.org/name")

# some relations among classes (not finished yet)
publicationVenue = URIRef("https://schema.org/isPartOf")

#base url for our subjects
base_url = "https://comp-data.github.io/res/"

Step 3: Create statements according to the dataframe (above) and load them into the graph.

In [5]:
#Load statements about publications:
#iterrows() is a method that allows us to select each row of the dataframe, one by one. Each row has its own index (idx)

for idx, row in graph_publications.iterrows():
    local_id = "publication-" + str(idx)
    
    # The shape of the new resources that are publications is
    # 'https://comp-data.github.io/res/publication-<integer>'
    subj = URIRef(base_url + local_id)
    
    #In this first case, the if condition tells us if each of the publications is of type "journal-article"
    #(look at the dataframe above)
    if row["type"] == "journal-article":
        my_graph.add((subj, RDF.type, JournalArticle))
    
        # These two statements applies only to journal article:
        my_graph.add((subj, issue, Literal(row["issue"])))
        my_graph.add((subj, volume, Literal(row["volume"])))
    else:
        my_graph.add((subj, RDF.type, BookChapter))
    my_graph.add((subj, name, Literal(row["title"])))
    my_graph.add((subj, publicationYear, Literal(row["publication_year"])))
        
    
    
    # The original value here has been casted to string since the Date type
    # in schema.org ('https://schema.org/Date') is actually a string-like value


So far, we have loaded a CSV file as a dataframe. We have then distinguished publications of type "journal-article" from publications of type "book-chapter". Each type of publications has its own attributes (issue and volume; title). 

Step 4: how many triples have we inserted in the graph?

In [6]:
len(my_graph)

2314

Step 5: create a Graph Database and populate it. In other words, we use the 1407(?) triples we have created so far to create a Graph database.

In [6]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

store = SPARQLUpdateStore()

# The URL of the SPARQL endpoint is the same URL of the Blazegraph
# instance + '/sparql'
endpoint = 'http://127.0.0.1:9999/blazegraph/sparql'

# It opens the connection with the SPARQL endpoint instance
store.open((endpoint, endpoint))

for triple in my_graph.triples((None, None, None)):
   store.add(triple)
    
# Once finished, remeber to close the connection
store.close()

Step 6: try to retrieve some data using SPARQL queries. In this case, we want to retrieve the <b>titles</b> (?title) of the publications having type <b>journal-article</b> (?journal_article). We need to recall two prefixes: the first one is the prefix referring to the property <code>rdf:type</code>; the second one contains both the object <code>schema:ScholarlyArticle</code> (i.e., the equivalent of the type Journal Article) and the property <code>schema:name</code>. 

In [7]:
from sparql_dataframe import get

endpoint = "http://127.0.0.1:9999/blazegraph/sparql"
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX schema: <https://schema.org/>

SELECT ?journal_article ?title
WHERE {
    ?journal_article rdf:type schema:ScholarlyArticle .
    ?journal_article schema:name ?title .
}
"""
df_sparql = get(endpoint, query, True)
df_sparql

,journal_article,title
0,https://comp-data.github.io/res/publication-486,Inferring User Interests In Microblogging Soci...
1,https://comp-data.github.io/res/publication-353,Enhanced Semantic Representation Of Coaxiality...
2,https://comp-data.github.io/res/publication-38,Ontology-Based Model To Support Ubiquitous Hea...
3,https://comp-data.github.io/res/publication-437,Relation Prediction In Knowledge Graph By Mult...
4,https://comp-data.github.io/res/publication-357,Webulous And The Webulous Google Add-On - A We...
...,...,...
402,https://comp-data.github.io/res/publication-407,"Sharing Biological Data: Why, When, And How"
403,https://comp-data.github.io/res/publication-339,"Integrated Modeling, Simulation, And Visualiza..."
404,https://comp-data.github.io/res/publication-208,Ontology-Based Annotation And Retrieval Of Ser...
405,https://comp-data.github.io/res/publication-392,Iddb: A Comprehensive Resource Featuring Genes...


In [8]:
endpoint = "http://127.0.0.1:9999/blazegraph/sparql"
query1 = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX schema: <https://schema.org/>

SELECT ?book_chapter ?title
WHERE {
    ?book_chapter rdf:type schema:Chapter .
    ?book_chapter schema:name ?title .
}
"""
df_sparql_2 = get(endpoint, query1, True)
df_sparql_2

,book_chapter,title
0,https://comp-data.github.io/res/publication-130,Sentiment Polarity Detection In Social Network...
1,https://comp-data.github.io/res/publication-88,Challenges In The Implementation Of Privacy En...
2,https://comp-data.github.io/res/publication-30,SantÃ©: A Light-Weight End-To-End Semantic Sea...
3,https://comp-data.github.io/res/publication-56,Se-Diagenf: An Ontology-Based Expert System Fo...
4,https://comp-data.github.io/res/publication-159,Comparative Study Of Rdf And Owl Ontology Lang...
...,...,...
88,https://comp-data.github.io/res/publication-31,Improving Answer Type Classification Quality T...
89,https://comp-data.github.io/res/publication-133,A Knowledge-Based Computational Environment Fo...
90,https://comp-data.github.io/res/publication-446,Enriching Wikidata With Cultural Heritage Data...
91,https://comp-data.github.io/res/publication-184,Mapping The Web Ontology Language To The Opena...


Step 7: Once we have created our database, we want to extract DataFrames from it. In order to do that, we need to define some classes with their own methods:

In [9]:
class TriplestoreProcessor(object):    
    def getEndpointUrl(self):
        return self.endpointUrl
    
    def setEndpointUrl(self, url):
        self.endpointUrl = url
        return True
        
#example
my_tripleP = TriplestoreProcessor()  
my_tripleP.setEndpointUrl("http://127.0.0.1:9999/blazegraph/sparql")
my_tripleP.getEndpointUrl()
 

'http://127.0.0.1:9999/blazegraph/sparql'

In [53]:
class TriplestoreQueryProcessor(TriplestoreProcessor):
    def getPublicationsPublishedInYear(self, year):
        endpoint = my_tripleP.getEndpointUrl()
        str_year = str(year)
        new_query = """
        PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX schema: <https://schema.org/>

        SELECT ?book_chapter ?title ?publicationYear
        WHERE {
            ?book_chapter rdf:type schema:Chapter .
            ?book_chapter schema:name ?title .
            ?book_chapter schema:datePublished ?publicationYear .
            FILTER (?publicationYear == 2019) 
        }
        """
        
        publ = get(endpoint, new_query.format(str_year), True)
        return publ
    
my_new_query = TriplestoreQueryProcessor()
my_new_query.getPublicationsPublishedInYear(2019)

KeyError: '\n            ?book_chapter rdf'